In [1]:
from first_stage_frlm import first_stage_frlm
from second_stage_frlm import second_stage_frlm
from visualize_placement import visualize_placement
import networkx as nx
import pickle
import matplotlib.pyplot as plt

In [2]:
G = pickle.load( open('data/network_digital_twin_v0.2.pickle','rb'))

In [3]:
a = nx.shortest_path(G,"8864461", "8860547", weight='length_m')
harbour_exits = pickle.load( open("data/berth_nodes.p", "rb") )

In [4]:
harbour_exits[:5]

Index(['Berth57', 'Berth51', 'Berth73', 'Berth184', 'Berth201'], dtype='object')

In [5]:
# Let's see whether the program works for the route from Brielle to Schwinhaven (near Zwijndrecht)
#now also added
df_b, df_g, df_eq_fq = first_stage_frlm(G, 40000, [("8864461", "8860547", 30)])

IndexError: list index out of range

In [ ]:
df_b

In [ ]:
df_g

In [ ]:
df_eq_fq

In [ ]:
optimal_facilities, optimal_flows = second_stage_frlm(6, 20, df_g, df_b, df_eq_fq)

In [ ]:
optimal_flows

## Seems to go quite okay!
Also try with multiple OD sets for illustration

In [ ]:
#now also added: Brielle to Middelharnis and Brielle to Ridderkerk
%time df_b, df_g, df_eq_fq = first_stage_frlm(G, 60000, [("8864461", "8860547", 30), ("8864461", "8862620", 20), ("8864461", "8860687", 10)])

In [ ]:
df_b

In [ ]:
df_g

In [ ]:
%time optimal_facilities, optimal_flows = second_stage_frlm(7, 20, df_g, df_b, df_eq_fq)

In [ ]:
optimal_facilities

In [ ]:
# Draw network, routes and charging stations

#Define new graph H with only nodes and edges in routes

OD_list = [("8864461", "8860547", 30), ("8864461", "8862620", 20), ("8864461", "8860687", 10)]

# OD_list = [("8864461", "8860547", 30)]#, ("8864461", "8862620", 20), ("8864461", "8860687", 10)]

# OD_list = [("8864991", "8863072", 10)]
node_list =[]
for origin, destination, flow in OD_list:
    for node in nx.dijkstra_path(G, origin, destination, weight='length_m'):
        if node not in node_list:
            node_list.append(node)

H = G.subgraph(node_list)

#create pos dict
pos_dict = {}
for node in node_list:
    pos_dict[node] = (G.nodes[node]['X'],G.nodes[node]['Y'])





other_ks =[]
fuel_stations = []

for key, number_of_stations in optimal_facilities.items():
    if number_of_stations > 0:
        fuel_stations.append(key)
    else:
        other_ks.append(key)


# x = nx.get_node_attributes(H, 'X')
# y = nx.get_node_attributes(H, 'Y')

# for key, item in x.items():
#     pos_dict[key] = (x[key],y[key])
# pos = (x,y)
    # print(pos)

# print(fuel_stations)
# H2 = H.subgraph[fuel_stations]
#
nx.draw_networkx_edges(H, pos = pos_dict)

nx.draw_networkx_nodes(G,pos_dict, fuel_stations, node_color='red')
nx.draw_networkx_nodes(G,pos_dict,other_ks, node_color='blue')
plt.show()

In [ ]:
OD_list = [("8864461", "8860547", 30), ("8864461", "8862620", 20), ("8864461", "8860687", 10)]


In [ ]:
visualize_placement(G, OD_list, optimal_facilities, True)